In [2]:
%load_ext autoreload
%autoreload 2

import IPython
IPython.get_ipython().run_line_magic('cd', '..')  # Go up one directory

from dictionary_learning.buffer import AllActivationBuffer
from dictionary_learning.trainers.top_k import TrainerSCAE, AutoEncoderTopK
from dictionary_learning.training import trainSCAE

from datasets import load_dataset
import torch as t
from nnsight import LanguageModel


device = "cuda:0" if t.cuda.is_available() else "cpu"
model = LanguageModel("gpt2", device_map=device)
model.eval()


dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/root


In [3]:
n_layer = model.config.n_layer

submodule_list = [
    (model.transformer.h[layer].mlp, "in_and_out")
    for layer in range(n_layer)
    ] 
submodule_list += [
    (model.transformer.h[layer].attn, "out")
     for layer in range(n_layer)
     ]

batch_size_tokens = 1024

buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodule_list,
    d_submodule=768, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower depending on your available memory
    device="cuda",
    out_batch_size = batch_size_tokens,
    refresh_batch_size = 256,
)  # buffer will yield batches of tensors of dimension = num_submodules * d_submodule


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [26]:
# First get the configuration from the buffer to set up our trainer properly
submodules = [submod for submod, _ in buffer.submodules]
activation_dims = [buffer.d_submodule[submod] for submod in submodules]
submodule_names = [submod.__class__.__name__ for submod in submodules]

# Configure the multi-trainer
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": activation_dims,
    "dict_sizes": [dim * 16 for dim in activation_dims],
    "ks": [128 for _ in submodules],  
    "device": buffer.device,
    "layers": list(range(len(submodules))),  # TODO: might want to fix this
    "lm_name": "your_model_name",  # replace with actual model name
    "submodule_names": submodule_names,
}
# Initialize trainer
trainer_class = trainer_cfg.pop("trainer")
trainer = trainer_class(**trainer_cfg)

In [27]:
xs, targets = next(buffer)

In [29]:
trainer.loss(xs)

tensor(135047.2344, device='cuda:0', grad_fn=<AddBackward0>)

In [18]:
# First get the configuration from the buffer to set up our trainer properly
submodules = [submod for submod, _ in buffer.submodules]
activation_dims = [buffer.d_submodule[submod] for submod in submodules]
submodule_names = [submod.__class__.__name__ for submod in submodules]

# Configure the multi-trainer
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": activation_dims,
    "dict_sizes": [dim * 16 for dim in activation_dims],
    "ks": [128 for _ in submodules],  
    "device": buffer.device,
    "layers": list(range(len(submodules))),  # TODO: might want to fix this
    "lm_name": "your_model_name",  # replace with actual model name
    "submodule_names": submodule_names,
}

# Run the training
trainer = trainSCAE(
    buffer=buffer,
    trainer_cfg=trainer_cfg,
    steps=30000,
    save_steps=1000,
    save_dir="sae_checkpoints",
    log_steps=100,
    use_wandb=True  # Set to False if you don't want to use wandb
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jacobcd52. Use `wandb login --relogin` to force relogin


  0%|          | 0/30000 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'shape'